Import Library

In [1]:
import numpy as np
import pandas as pd

c:\Users\jihan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\jihan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.4SP5SUA7CBGXUEOC35YP2ASOICYYEQZZ.gfortran-win_amd64.dll
c:\Users\jihan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\jihan\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Membuat kelas untuk LDA

In [ ]:
class LDA:
    def __init__(self, n_components):
        self.n_components = n_components
        self.linear_discriminants = None

    def fit(self, X, y):
        n_features = X.shape[1]
        class_labels = np.unique(y)

        # Rumus untuk Within class scatter matrix:
        # SW = sum((X_c - mean_X_c)^2 )

        # Rumus untuk Between class scatter:
        # SB = sum( n_c * (mean_X_c - mean_overall)^2 )

        mean_overall = np.mean(X, axis=0) 
        # print("Mean Overall: ", mean_overall)
        SW = np.zeros((n_features, n_features)) #Membuat dimensi matriks untuk SW
        SB = np.zeros((n_features, n_features))
        for c in class_labels:
            X_c = X[y == c]
            # print("x_c: \n",X_c)
            mean_c = np.mean(X_c, axis=0)
            # (4, n_c) * (n_c, 4) = (4,4) -> transpose
            minus = X_c - mean_c
            SW += (minus).T.dot((X_c - mean_c))
            # print("Minus:", minus)
            # print("SW1:",SW)

            # (4, 1) * (1, 4) = (4,4) -> reshape
            n_c = X_c.shape[0]
            print("n_c",n_c)
            mean_diff = (mean_c - mean_overall).reshape(n_features, 1)
            print("Mean Diff: \n", mean_diff)
            SB += n_c * (mean_diff).dot(mean_diff.T)
            print("SB dalem: ",SB)
            print("Mean C: \n", mean_c)

        
        print("SW: ", SW)
        print("SB: ", SB)

        # Determine SW^-1 * SB
        A = np.linalg.inv(SW).dot(SB)
        print("A:", A)
        # Get eigenvalues and eigenvectors of SW^-1 * SB
        eigenvalues, eigenvectors = np.linalg.eig(A)
        # -> eigenvector v = [:,i] column vector, transpose for easier calculations
        # sort eigenvalues high to low
        eigenvectors = eigenvectors.T
        idxs = np.argsort(abs(eigenvalues))[::-1]
        eigenvalues = eigenvalues[idxs]
        eigenvectors = eigenvectors[idxs]
        print("Eigen Vector:\n",eigenvectors)
        print("Eigen Values\n", eigenvalues)
        # store first n eigenvectors
        self.linear_discriminants = eigenvectors[0 : self.n_components]

    def transform(self, X):
        # project data
        return np.dot(X, self.linear_discriminants.T)